In [275]:
import requests
import string
import pandas as pd
from bs4 import BeautifulSoup
import datetime 
from dateutil.parser import parse
from datetime import timedelta

url: https://forecast.weather.gov/MapClick.php?lat=37.777120000000025&lon=-122.41963999999996#.YFsugpzPyiM


inspect with the console
write code to target the right elements of the text

For each day, you need to store:

-The day of the week (Saturday)

-The date (12/09)

-A short description of the conditions (Clear early then increasing cloudiness after midnight. Low 41F. Winds light and variable)

-The temperature low and high, converted into Celsius

-For each element you scrape, The name of the item you targetted 
(ex: DailyContent--daypartDate--3MM0J)

-Save all of this into a Pandas dataframe


In [277]:
# Weather.gov

page = requests.get("https://forecast.weather.gov/MapClick.php?lat=37.777120000000025&lon=-122.41963999999996#.YFsugpzPyiM")
soup = BeautifulSoup(page.content, 'html.parser')

mydivs = soup.find_all("div", {"id": "seven-day-forecast-container"})[0]
#print(mydivs)

#target parents
#go deeper into childen
#target
##<div id="seven-day-forecast-container"><ul class="list-unstyled" id="seven-day-forecast-list"><li class="forecast-tombstone">

#find p element or relevant class

    #target day
    #target temperature
    #target summary 
my_p = mydivs.find_all("p")
#my_p = mydivs.find_all("p", {"class":"period-name"})

#save into a list
my_p_list=[]
for el in my_p:
    my_p_list.append(el.get_text())
    #print(el,el.get_text())
#Excluding Today
my_p_list = my_p_list[8:]
#my_p_list = my_p_list.split()
my_p_list = [my_p_list[i:i+8] for i in range(0,len(my_p_list),8)]
#print(my_p_list)

#save into a dictionary?
#{"day1":["sunday", "54", 'today was a great sunny day '], "day2":["monday", "54", 'today was a great sunny day ']}
mydict = {}
i=1
#Find date
today = soup.find_all("div", {"id":"current_conditions_detail"})[0].find_all("td")[-1].get_text().strip()

for ele in my_p_list:

    date = parse(today).date()+timedelta(days=i)
    
    mydict[f"Day{i}"] = [date, ele[0],f"{ele[2]} and later {ele[-2]}",
    str(round((float(ele[-1][-5:-3])-32)*5/9,2))+"C",
    str(round((float(ele[3][-5:-3])-32)*5/9,2))+"C"]
    
    i += 1
#print(mydict)


#load dict into pandas
mydf = pd.DataFrame(mydict, index=["Date", "Day", "Description","Low in °C","High in °C"])
mydf

# create column names if needed






/shared-libs/python3.7/py-core/lib/python3.7/site-packages/dateutil/parser/_parser.py:1218: UnknownTimezoneWarning: tzname PDT identified but not understood.  Pass `tzinfos` argument in order to correctly return a timezone-aware datetime.  In a future version, this will raise an exception.
  category=UnknownTimezoneWarning)


,Day1,Day2,Day3,Day4
Date,2021-03-25,2021-03-26,2021-03-27,2021-03-28
Day,Thursday,Friday,Saturday,Sunday
Description,Sunny and later Clear,Sunny and later Clear,Sunny and later Clear,Sunny and later Sunny
Low in °C,7.78C,8.33C,9.44C,20.56C
High in °C,16.11C,20.56C,23.33C,20.56C


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=79673256-d468-4dc3-bcd4-af2c4cf3525d' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>